In [3]:
import time
!pip install pandas
import sys
from dotenv import load_dotenv
import pandas as pd
import os
import google.generativeai as genai
from IPython.display import Markdown, display,clear_output
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

data=[
    {"prompt":"Write a python code to calculate factorial of a number",
     "response": "def fact(n):\n    return n*fact(n-1)"},
    {"prompt":"Write python code to calculate fibonacci number",
     "response":"def fib(n):\n  return fibo(n-1)+fibo(n-2)"
    },
    {"prompt":"Write python code to calcuate log 2 of a function ",
     "response":"def logof2(n):\ncount=0   while(n):\n   if(n&1==1):\n    count+=1\nn=n&1",
    }
]

df=pd.DataFrame(data)
df.to_csv("code_generation_dataset.csv",index=False)
df.head()

,prompt,response
0,Write a python code to calculate factorial of ...,def fact(n):\n return n*fact(n-1)
1,Write python code to calculate fibonacci number,def fib(n):\n return fibo(n-1)+fibo(n-2)
2,Write python code to calcuate log 2 of a funct...,def logof2(n):\ncount=0 while(n):\n if(n&1...


In [16]:
import json
train_data=df.sample(frac=0.7,random_state=1)
test_data=df.drop(train_data.index)
def savejsonlfile(df,filename):
    with open(filename,'w') as f:
        for _,row in df.iterrows():
            entry={
                'messages':[
                {'role':'user', 'content':row['prompt']},
                {'role':'assistant','content':row['response']}
                ]
            }
            f.write(json.dumps(entry)+'\n')

savejsonlfile(train_data,'traindata.jsonl')
savejsonlfile(test_data,'testdata.jsonl')

In [19]:
load_dotenv()  # This loads the .env file
api_key = os.getenv("GEMINI_API_KEY")

genai.configure(api_key=api_key)
model=genai.GenerativeModel('gemini-2.0-flash')

def generateresponse(prompt):
    try:
        response1=model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.2,
                top_p=1,
                top_k=1,
                max_output_tokens=180
            ))
        return response1.text
    except Exception as e:
        return f"Error:{str(e)}"
    
prompt = "Write a Python function to find if a string is palindrome"
print(generateresponse(prompt))

```python
def is_palindrome(text):
  """
  Checks if a given string is a palindrome (reads the same forwards and backward).

  Args:
    text: The string to check.

  Returns:
    True if the string is a palindrome, False otherwise.  Ignores case and non-alphanumeric characters.
  """
  processed_text = ''.join(char.lower() for char in text if char.isalnum())
  return processed_text == processed_text[::-1]

# Example usage:
print(is_palindrome("racecar"))  # Output: True
print(is_palindrome("A man, a plan, a canal: Panama"))  # Output: True
print(is_palindrome("hello"))  # Output: False
print(is_palindrome("Madam")) #


In [11]:
from dotenv import load_dotenv
import os
load_dotenv()  # This loads the .env file
api_key = os.getenv("GEMINI_API_KEY")

genai.configure(api_key=api_key)
model=genai.GenerativeModel('gemini-2.0-flash')

def optimize_code(prompt):
    sysprompt="Optimize the given python to increase its speed, and readability"
    try:
        response1=model.generate_content(
            sysprompt+" "+prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.3,
                max_output_tokens=600,
                top_p=0.7,
                top_k=40
            )
        )
        return response1.text
    except Exception as e:
        return f"Error:{str(e)}"

prompt1=input("Enter code: ")
print(optimize_code(prompt1))


```python
def fib_optimized(n):
  """
  Calculates the nth Fibonacci number using memoization (dynamic programming).

  This version is significantly faster than a naive recursive implementation,
  especially for larger values of n, due to avoiding redundant calculations.

  Args:
    n: The index of the desired Fibonacci number (non-negative integer).

  Returns:
    The nth Fibonacci number.  Returns 0 if n is 0.

  Raises:
    TypeError: if n is not an integer.
    ValueError: if n is negative.
  """

  if not isinstance(n, int):
    raise TypeError("Input must be an integer.")
  if n < 0:
    raise ValueError("Input must be a non-negative integer.")

  if n <= 1:
    return n  # Base cases: F(0) = 0, F(1) = 1

  # Use memoization to store calculated Fibonacci numbers
  memo = [0] * (n + 1)
  memo[0] = 0
  memo[1] = 1

  for i in range(2, n + 1):
    memo[i] = memo[i - 1] + memo[i - 2]

  return memo[n]


# Example Usage:
print(fib_optimized(10))  # Output: 55
print(fib_optimized(0)

In [13]:
def debug_code(prompt):
    sysprompt="Define and solve the error in this code "
    try:
        response1=model.generate_content(
            sysprompt+" "+prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.3,
                max_output_tokens=600,
                top_p=0.7,
                top_k=40
            )
        )
        return response1.text
    except Exception as e:
        return f"Error:{str(e)}"

prompt1=input("Enter code: ")
print(debug_code(prompt1))

```python
def fact(n):
  """
  Calculates the factorial of a non-negative integer.

  Args:
    n: A non-negative integer.

  Returns:
    The factorial of n (n!), or 1 if n is 0.  Returns None if n is negative.
  """
  if n < 0:
    return None  # Factorial is not defined for negative numbers
  elif n == 0:  # Base case for n=0
    return 1
  else:
    return n * fact(n - 1)

# Example Usage and Testing
print(fact(5))  # Output: 120
print(fact(0))  # Output: 1
print(fact(1))  # Output: 1
print(fact(3))  # Output: 6
print(fact(-1)) # Output: None
```

**Error Definition and Solution:**

1. **Missing Base Case for n=0:** The original code only had a base case for `n == 1`.  The factorial of 0 (0!) is defined as 1.  Without this base case, if you call `fact(0)`, the recursion would continue indefinitely (or until it hits the recursion limit) because it would keep calling `fact(-1)`, `fact(-2)`, and so on.  This leads to a `RecursionError`.

2. **No Handling of Negative Input:** Factorial

In [16]:
def check_code(prompt):
    sysprompt="Check this code, if there is any error find and name the error, give back corrected code, next give an optimized version of code"
    try:
        response1=model.generate_content(
            sysprompt+" "+prompt,
            generation_config=genai.GenerationConfig(
                max_output_tokens=550,
                temperature=0.3,
                top_k=40,
                top_p=0.7
            ))
        return response1.text
     
    except Exception as e:
        return f"error: {str(e)}"

inpText=input("Enter the code: ")
print(check_code(inpText))
    

Okay, let's analyze the Python code you provided.

**1. Identifying the Error**

The primary error in the code is the potential for a `ZeroDivisionError`.  If `b` is zero, the division operation `a / b` will raise this exception, causing the program to crash.

**2. Corrected Code**

Here's the corrected code, including error handling:

```python
def div(a, b):
    """Divides a by b, handling potential ZeroDivisionError."""
    if b == 0:
        return "Error: Cannot divide by zero"  # Or raise a more specific exception
    else:
        c = a / b
        return c
```

**Explanation of Correction:**

*   **Error Handling:**  The `if b == 0:` statement checks if the divisor (`b`) is zero.
*   **Return Value on Error:** If `b` is zero, the function now returns an error message (a string).  Alternatively, you could raise a `ValueError` or `ZeroDivisionError` explicitly using `raise ZeroDivisionError("Cannot divide by zero")`.  Raising an exception is often preferred because it forces the 